In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
import warnings
import utils
warnings.filterwarnings("ignore")

In [2]:
weather = pd.read_csv('../raw_data/weather.csv', index_col=0)

train_df = pd.concat([pd.read_csv('../raw_data/yellow_tripdata_2018-06.csv'),
                      pd.read_csv('../raw_data/yellow_tripdata_2018-07.csv')])
train_X, train_y, onehoe_enc, scaler = utils.preprocessing(train_df, weather, [2018, 6, 1], [2018, 7, 31])
sparse.save_npz('../preprocessed_data/train_X.npz', train_X)
np.save('../preprocessed_data/train_y.npy', train_y)

valid_df = pd.read_csv('../raw_data/yellow_tripdata_2018-08.csv')
valid_X, valid_y, _, _ = utils.preprocessing(valid_df, weather, [2018, 8, 1], [2018, 8, 31], onehoe_enc, scaler)
sparse.save_npz('../preprocessed_data/valid_X.npz', valid_X)
np.save('../preprocessed_data/valid_y.npy', valid_y)

test_df = pd.read_csv('../raw_data/yellow_tripdata_2018-09.csv')
test_X, test_y, _, _ = utils.preprocessing(test_df, weather, [2018, 9, 1], [2018, 9, 30], onehoe_enc, scaler)
sparse.save_npz('../preprocessed_data/test_X.npz', test_X)
np.save('../preprocessed_data/test_y.npy', test_y)


In [3]:
train_X.shape

(9430744, 573)

In [4]:
# discrete attribute

attribute_dic = {}
for i, j in enumerate(utils.columns_discrete):
    attribute_dic['x' + str(i)] = j
attribute_name = pd.DataFrame(onehoe_enc.get_feature_names(), columns=['attribute_name'])
attribute_name['name'] = attribute_name['attribute_name'].map(lambda x: x.split('_')[0])
attribute_name['name'] = attribute_name['name'].map(lambda x: attribute_dic[x])
attribute_name['category'] = attribute_name['attribute_name'].map(lambda x: x.split('_')[1])
attribute_name = attribute_name[['name', 'category']]

# continuous attribute
attribute_name = attribute_name.append(pd.DataFrame([[i, ''] for i in utils.columns_continuous],
                                                    columns=['name', 'category']))

attribute_name.to_csv('../preprocessed_data/attribute_dic.csv')
attribute_name


,name,category
0,PULocationID,1
1,PULocationID,2
2,PULocationID,3
3,PULocationID,4
4,PULocationID,5
...,...,...
0,passenger_count,
1,trip_distance,
2,fare_amount,
3,tip_amount,
